# Sire diaries: Working with OpenMM

[OpenMM](https://openmm.org) is a fantastic high performance toolkit for molecular simulation. It is perhaps best known for providing a flexible Python scripting environment for defining and running custom GPU-accelerated molecular dynamics simulations.

In the [2023.2 release of sire](https://sire.openbiosim.org) we added new [sire.convert](https://sire.openbiosim.org/tutorial/index_part05.html) functions. In the [last post](https://www.openbiosim.org/working-with-rdkit/) we showed how these functions could be used to interconvert with [RDKit](https://www.rdkit.org), thereby adding both [smiles](https://sire.openbiosim.org/tutorial/part05/03_smiles.html) and [2D visualisation support](https://sire.openbiosim.org/tutorial/part05/02_view.html).

In this post, we will show how [sire.convert](https://sire.openbiosim.org/tutorial/index_part05.html) can be used to interconvert with [OpenMM](https://openmm.org). This lets sire be used as an interface for setting up OpenMM simulations. We have also added a higher-level [set of functions](https://sire.openbiosim.org/cheatsheet/openmm.html#higher-level-api) that lets you easily run minimisation and molecular dynamics simulations directly from a set of sire molecules.

For example, here, we will load a molecular system, perform minimisation, and then run molecular dynamics.

In [ ]:
import sire as sr

# Load the molecules from our "tutorial" website
mols = sr.load(
    sr.expand(sr.tutorial_url, "ala.top", "ala.crd"), silent=True)

# Perform minimisation
mols = mols.minimisation().run().commit()

# Now perform 10 ps of dynamics, saving a snapshot every 0.5 ps
mols = mols.dynamics(
            timestep=4*sr.units.femtosecond
        ).run(10*sr.units.picosecond,
              save_frequency=0.5*sr.units.picosecond).commit()

We can view a movie of the resulting trajectory using our [integration with NGLView](https://sire.openbiosim.org/cheatsheet/view.html#id1).

In [ ]:
mols.view()

We can plot energies of interaction, e.g. between the solute and the water for each of the saved frame of this trajectory using

In [ ]:
mols[0].trajectory().energy(mols["water"]).pretty_plot()

This trajectory is, admittedly, very short, on a small molecular system. But the same code would work for larger systems over longer timescales. We just needed something here that you could run easily on our [free notebook server](https://try.openbiosim.org) ;-)

Under the hood, this works because [sire.convert](https://sire.openbiosim.org/cheatsheet/openmm.html) has been called to create an OpenMM system from the loaded sire molecules. This was then manipulated using the standard OpenMM functions to run minimisation and dynamics.

For example, lets create the OpenMM system from our molecules.

In [ ]:
omm = sr.convert.to(mols, "openmm")
omm

The result is an [openmm.Context](https://docs.openmm.org/latest/api-python/generated/openmm.openmm.Context.html#openmm.openmm.Context). This contains a combination of an [openmm.System](https://docs.openmm.org/latest/api-python/generated/openmm.openmm.System.html#openmm.openmm.System), [openmm.Integrator](https://docs.openmm.org/latest/api-python/library.html#integrators) and [openmm.Platform](https://docs.openmm.org/latest/api-python/generated/openmm.openmm.Platform.html#openmm.openmm.Platform).

You can extract these three object using the standard OpenMM functions, e.g.

In [ ]:
omm_system = omm.getSystem()
omm_system

In [ ]:
omm_integrator = omm.getIntegrator()
omm_integrator

In [ ]:
omm_platform = omm.getPlatform()
omm_platform

You can combine the `omm_system` with your own choices of integrator or platform using the standard OpenMM API, if you don't like the choices made by sire.

To keep the script simple, sire chooses appropriate default settings depending on the molecular system being modelled and the requested simulation timestep. You have complete control of these defaults, and can override any of them using a simple [set of options](https://sire.openbiosim.org/cheatsheet/openmm.html#choosing-options).

The way you do this is by manipulating the [Dynamics](https://sire.openbiosim.org/api/mol.html#sire.mol.Dynamics) object which is returned by the `.dynamics()` function.

In [ ]:
d = mols.dynamics(timestep=4*sr.units.femtosecond)
d

You can query the current options by calling the `.info()`, `.constraint()` and `.ensemble()` functions

In [ ]:
d.info(), d.constraint(), d.ensemble()

In this case, you can see that sire has detected that the molecules are in a periodic box, and so has switched on PME boundary conditions and set a cutoff of 7.5 Å. 

It has seen that you haven't specified a temperature or pressure, so it chooses an integrator appropriate for the microcanonical (NVE) ensemble.

And it uses your choice of a larger timestep (4 fs) to automatically add constrains to all bonds, and to all angles involving hydrogen (the `bonds-h-angles` constraints).

You can easily change these options, by passing in arguments as described in our [detailed guide](https://sire.openbiosim.org/cheatsheet/openmm.html#choosing-options).

In [ ]:
d = mols.dynamics(timestep=1*sr.units.femtosecond,
                  map={"cutoff": 10*sr.units.angstrom,
                       "temperature": 25*sr.units.celsius,
                       "cutoff_type": "REACTION_FIELD"})

d.info(), d.constraint(), d.ensemble()

In this case, we have now switched to a 10 Å reaction field cutoff, and have an integrator that is suited for the canonical (NVT) ensemble at 25°C. Bond and angle constraints have been automatically disabled as a small timestep is being used (you can force a choice by setting the `constraint` option).

We can then run the simulation by calling `d.run()` passing in the amount of time we want to simulate, and the amount of time between saving snapshots. Sire then automatically works out the required number of simulation steps based on this, and the desired timestep.

In [ ]:
d.run(2*sr.units.picosecond, 
      save_frequency=0.1*sr.units.picosecond)

We can call this `.run()` function as often as we want, changing the amount of time to simulate and the `save_frequency` as we desire. The coordinates will be appended to our now ever-growing trajectory.

In [ ]:
d.run(5*sr.units.picosecond,
      save_frequency=1*sr.units.picosecond)

To finish, we commit this simulation back, converting it back into a sire molecular system. This can be viewed or processed, just as if it had been loaded from a file.

In [ ]:
mols = d.commit()
mols.view()

In [ ]:
mols[0].trajectory().energy(mols["water"]).pretty_plot()

Notice how the different values of `save_frequency` for our there trajectories appear differently in the movie and in the above energy graph.

You are not limited to simulating all the molecules. You can simulate any molecule. For example, here we will run MD on the solute (first molecule) loaded from another file.

In [ ]:
mols = sr.load(sr.expand(sr.tutorial_url, 
                         "ala.top", "ala.crd"), silent=True)
mol = mols[0].dynamics(
                timestep=4*sr.units.femtosecond
            ).run(
                10*sr.units.picosecond,
                save_frequency=0.1*sr.units.picosecond
            ).commit()

In [ ]:
mol.view()

In [ ]:
mol.trajectory().energy().pretty_plot()

This is just the beginning of what we have planned using our integration with [OpenMM](https://openmm.org). We have better trajectory support which will be part of our `2023.3.0` release later this month, and plans for even more interesting functionality that should be ready for `2023.4.0` later this year. Keep an eye on our [GitHub repo](https://github.com/openbiosim/sire) if you want to know more ;-)

As with all of our diaries, if you want to try this yourself, please feel free connect to [try.openbiosim.org](https://try.openbiosim.org) and starting a notebook. You can download the notebook used to generate this post onto the server by running this command in one of the notebook code cells.

! wget https://github.com/OpenBioSim/posts/raw/main/sire/003_openmm/openmm.ipynb

Have a play and [let us know what you think](https://github.com/OpenBioSim/sire/issues). Look forward to our next blog post, where we will show you how to use the new code that will be in `2023.3.0` to load, save, edit and convert molecular trajectories.